### Get the fixtures and results of the big 5 european leagues for the last 5 years in the following data format
| Match Date | Home Team | Away Team | Score | Result    | XG_Home | XG_Away | League | Year | match_report_url |
|------------|-----------|-----------|-------|-----------|---------|---------|--------|------|------------------|
| Date 1     | Team 1    | Team 2    | 1-1   | Draw      | 1.1     | 1.2     | EPL    | 2022 | url1             |
| Date 2     | Team 2    | Team 3    | 2-0   | Team2-Win | 1.2     | 0.22    | EPL    | 2022 | url2             |
|            |           |           |       |           |         |         |        |      |                  |

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from typing import Dict
from tqdm import tqdm

First, get the starting URL for all the 5 leagues.

EPL = https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures </br>
Serie A = https://fbref.com/en/comps/11/schedule/Serie-A-Scores-and-Fixtures </br>
LaLiga = https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures </br>
Bundesliga = https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures </br>
Ligue 1 = https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures </br>


Construct the URL for last 6 years. (2022-2023, 2021-2022, 2020-2021, 2019-2020, 2018-2019, 2017-2018) </br>
For all the 5 leagues, we would have (6*5 = 30), 30 urls in total.


In [ ]:
def create_url_configs(year:str):
  # Use the year to construct a dictionary of url configs for all the stats.
  config =  {
    "EPL": "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures" if year is None else f"https://fbref.com/en/comps/9/{year}/schedule/{year}-Premier-League-Scores-and-Fixtures",
    "SERIE_A": "https://fbref.com/en/comps/11/schedule/Serie-A-Scores-and-Fixtures" if year is None else f"https://fbref.com/en/comps/11/{year}/schedule/{year}-Serie-A-Scores-and-Fixtures",
    "LALIGA": "https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures" if year is None else f"https://fbref.com/en/comps/12/{year}/schedule/{year}-La-Liga-Scores-and-Fixtures",
    "BUNDESLIGA": "https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures" if year is None else f"https://fbref.com/en/comps/20/{year}/schedule/{year}-Bundesliga-Scores-and-Fixtures",
    "LIGUE_1": "https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures" if year is None else f"https://fbref.com/en/comps/13/{year}/schedule/{year}-Ligue-1-Scores-and-Fixtures",
    }
  return config

For each URL, writer a parser that would return the above dataframe.

In [ ]:
def parse_page(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    #go to the first table of the page
    table = soup.findAll('table')[0]

    #list of all columns of the table & remove gamewee
    features = [col.attrs["data-stat"] for col in table.find('thead').findAll('tr')[0].findAll('th')]
    features.remove("gameweek")

    #Parse table body
    match_table = table.find('tbody')
    pre_df_match = []
    rows_match = match_table.find_all('tr')


    for row in rows_match:
        # avoid scraping spacer rows
        if("class" not in row.attrs):
            data = [x.text.strip().encode().decode("utf-8") for x in row.find_all('td')]

            parsed = dict(zip(features, data))
            parsed_removed = remove_features(parsed)

            parsed_removed["match_report_url"] = get_match_report_url(row)

            pre_df_match.append(parsed_removed)


    df_match = pd.DataFrame(pre_df_match, columns= pre_df_match[0].keys())
    return df_match

In [ ]:
def remove_features(row: Dict) -> Dict:
  row = row.copy()
  for stat in ["dayofweek", "start_time", "attendance", "venue", "referee", "match_report", "notes"]:
    del row[stat]
  return row

In [ ]:
def get_match_report_url(row):
  report = [x for x in row.find_all("td") if ("data-stat" in x.attrs) and (x.attrs["data-stat"] == "match_report")]
  if report is not None and len(report) == 1:
    a_link = report[0].find('a')
    if a_link is not None:
      return "https://fbref.com/" + a_link.get('href')
  return ''

In [ ]:
def clean_manager_name(element):
  manager_element = [x for x in element.find_all('div', class_='datapoint') if x.find('strong').text.lower() == "manager"]
  if manager_element is not None and len(manager_element) == 1:
    return ' '.join(manager_element[0].text.split(':')[1].strip().split('\xa0'))
  return ''

In [ ]:
def parse_match_report_url(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content)
  scorebox = soup.find('div', class_='scorebox')
  all_divs = scorebox.findAll('div')

  home_team = all_divs[0]
  away_team = all_divs[10]
  assert len(home_team.findAll('div')) == 9
  assert len(away_team.findAll('div')) == 9
  home_manager_name = clean_manager_name(home_team)
  away_manager_name = clean_manager_name(away_team)
  return {
      "url": url,
      "home_manager": home_manager_name,
      "away_manager": away_manager_name
  }

Once, done, just concatenate the dataframe and store it in a csv.

In [ ]:
all_years = [None, "2021-2022", "2020-2021", "2019-2020", "2018-2019", "2017-2018"]
all_years_df = []
for year in tqdm(all_years):
  url_configs = create_url_configs(year)
  for league in url_configs.keys():
    league_year_url = url_configs[league]
    df = parse_page(league_year_url)
    df["league"] = league
    df["year"] = "CURRENT" if year is None else year
    all_years_df.append(df)
full_df = pd.concat(all_years_df)
full_df.to_csv('./big5_last_6_years_result.csv', index=False)

100%|██████████| 6/6 [01:01<00:00, 10.20s/it]


In [ ]:
full_df.sort_values(['date'], ascending=False).head()

,date,home_team,home_xg,score,away_xg,away_team,match_report_url,league,year,round
307,2023-06-05,Hamburger SV,,1–3,,Stuttgart,https://fbref.com//en/matches/f9a47a86/Hamburg...,BUNDESLIGA,CURRENT,
372,2023-06-04,Real Madrid,1.8,1–1,2.3,Athletic Club,https://fbref.com//en/matches/ce677ae3/Real-Ma...,LALIGA,CURRENT,NaN
379,2023-06-04,Espanyol,0.6,3–3,1.8,Almería,https://fbref.com//en/matches/9a77cbfa/Espanyo...,LALIGA,CURRENT,NaN
374,2023-06-04,Napoli,2.0,2–0,0.6,Sampdoria,https://fbref.com//en/matches/971ace55/Napoli-...,SERIE_A,CURRENT,NaN
375,2023-06-04,Atalanta,4.0,5–2,1.2,Monza,https://fbref.com//en/matches/08c664e0/Atalant...,SERIE_A,CURRENT,NaN


### Use the match report urls to get the manager names and other details if needed.

In [ ]:
import os
manager_df_partial = pd.read_csv('./manager_df_partial.csv') if 'manager_df_partial.csv' in os.listdir('./') else None
existing_match_reports = list(manager_df_partial['url']) if manager_df_partial is not None else []

In [ ]:
urls_to_process = set(full_df['match_report_url']).difference(set(existing_match_reports))

In [ ]:
from time import sleep
manager_names = []
for url in tqdm(urls_to_process):
  try:
    if url is not None and url!= '':
      if url.strip() in list(existing_match_reports):
        manager_names.append(manager_df_partial[manager_df_partial.url == url].to_dict())
      else:
        manager_names.append(parse_match_report_url(url))
        sleep(3)
  except Exception as e:
    print(f'Error processing {url}')


 52%|█████▏    | 14/27 [00:55<00:41,  3.22s/it]

Error processing https://fbref.com//en/matches/f9a47a86/Hamburger-SV-Stuttgart-June-5-2023-German-12-RelegationPromotion-Play-offs


 96%|█████████▋| 26/27 [01:44<00:03,  3.10s/it]

Error processing https://fbref.com//en/matches/2c791569/Stuttgart-Hamburger-SV-June-1-2023-German-12-RelegationPromotion-Play-offs


100%|██████████| 27/27 [01:47<00:00,  3.99s/it]


In [ ]:
unprocessed_df = pd.DataFrame(manager_names, columns = manager_names[0].keys())
manager_df = pd.concat([manager_df_partial, unprocessed_df])
manager_df.to_csv('./manager_df_full.csv', index=False)